## 3D CNN on ShapeNetCore (5 classes) — step‑by‑step notebook

Goal: Train a simple 3D CNN classifier on a 5‑class subset of ShapeNetCore using the provided model_normalized.surface.binvox voxel files. This notebook is written to be learnable: bite‑sized steps, clear headings, and plenty of comments.

In [ ]:
from pathlib import Path
import json, pkgutil, io

# ✍️ Set this to your local dataset folder (the directory that contains the synset folders, e.g. 03211117, 02808440, …)
SHAPENET_PATH = Path('../data/shapeNetCore').expanduser().resolve()
assert SHAPENET_PATH.exists(), f"ShapeNetCore folder not found at {SHAPENET_PATH}"
print("Using SHAPENET_PATH:", SHAPENET_PATH)

import torch
device = (
    torch.device("mps") if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available()
    else torch.device("cuda") if torch.cuda.is_available()
    else torch.device("cpu")
)
device

Using SHAPENET_PATH: /Users/brageramberg/Desktop/3DCNN/data/shapeNetCore


device(type='mps')

## .binvox reader

In [3]:
import numpy as np
import gzip
import struct

def read_binvox(path):
    """
    Minimal binvox reader that returns a boolean numpy array (D, H, W).
    Supports RLE 'data' format used by ShapeNet binvox files.
    """
    # open file (supports gz too)
    open_fn = gzip.open if str(path).endswith(".gz") else open
    with open_fn(path, "rb") as f:
        # header
        line = f.readline().decode("ascii").strip()
        if line != "#binvox 1":
            raise ValueError("Not a binvox file")
        dims = []
        translate = []
        scale = None
        while True:
            line = f.readline().decode("ascii").strip()
            if line.startswith("dim"):
                _, dx, dy, dz = line.split()
                dims = (int(dx), int(dy), int(dz))
            elif line.startswith("translate"):
                translate = list(map(float, line.split()[1:]))
            elif line.startswith("scale"):
                scale = float(line.split()[1])
            elif line == "data":
                break

        # RLE voxels (value, count) pairs of unsigned bytes
        raw = f.read()
        # two bytes per run
        values = np.frombuffer(raw, dtype=np.uint8)
        assert len(values) % 2 == 0, "Corrupt binvox RLE stream"
        values = values.reshape(-1, 2)
        # expand runs
        voxels = np.repeat(values[:, 0], values[:, 1]).astype(np.uint8)
        voxels = voxels.reshape(dims)  # dims order: x, y, z
        # binvox stores as x,y,z — we’ll standardize to z,y,x (D,H,W) for PyTorch (channels-first later)
        voxels = np.transpose(voxels, (2, 1, 0))  # (z, y, x)
        voxels = voxels.astype(bool)
        return voxels